In [1]:
import json
import requests
import quandl
import pandas as pd
import datetime

from pandas.io.json import json_normalize



response = requests.get("https://min-api.cryptocompare.com/data/histoday?fsym=ETH&tsym=USD&allData=TRUE")


data = json.loads(response.text)
data2 = json_normalize(data)
json_data = data['Data']
df=pd.DataFrame(data['Data'])
df['Date'] = pd.to_datetime(df['time'],unit='s')
df.dtypes
df.astype(object)

Eth = df[['Date','open','high','low']]
Eth['ETHOpen'] = Eth['open']
Eth['ETHHigh'] = Eth['high']
Eth['ETHLow'] = Eth['low']
Eth = Eth[['Date','ETHOpen','ETHHigh','ETHLow']]

# US Treasury Bond Futures
USTBonds = quandl.get('CHRIS/CME_US1')

# US Dollar Index Futures
USDIndexFutures = quandl.get('CHRIS/ICE_DX2')
    
# Inflation YOY - USA
USInflationRate = quandl.get('RATEINF/INFLATION_USA')

# Unemployment Level
USUnemp = quandl.get('FRED/UNRATE')
    
# Effective Federal Funds Rate (Interest Rate)
USFedFundRate = quandl.get('FRED/DFF')

# United States GDP at Current Prices, LCU Billions
USGDP = quandl.get('ODA/USA_NGDP')




D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [157]:


USTBonds = pd.DataFrame(USTBonds)
USTBonds['Date'] = USTBonds.index.values
TBonds = USTBonds[['Date','Open']]
TBonds = TBonds.rename(columns = {'Open':'TBondsOpenValue'})

USDIndexFutures = pd.DataFrame(USDIndexFutures)
USDIndexFutures['Date'] = USDIndexFutures.index.values
IndexFutures = USDIndexFutures[['Date','Open']]
IndexFutures['IndexFutures'] = IndexFutures['Open']
IndexFutures = IndexFutures[['Date','IndexFutures']]

USFedFundRate = pd.DataFrame(USFedFundRate)
USFedFundRate['Date'] = USFedFundRate.index.values
FedFundRate = USFedFundRate[['Date','Value']]
FedFundRate['FedFundRateValue'] = FedFundRate['Value']
FedFundRate = FedFundRate[['Date','FedFundRateValue']]

USInflationRate = pd.DataFrame(USInflationRate)
USInflationRate['Date'] = USInflationRate.index.values
USInflationRate.insert(0, 'NumInd', range(0, 0 + len(USInflationRate)))
USInflationRate.set_index('NumInd', inplace=True)
USInflationRate['start'] = USInflationRate['Date']
USInflationRate['end'] = USInflationRate['Date']
USInflationRate.end = USInflationRate.end.shift(-1)
USInflationRate.end = USInflationRate['end'].fillna(datetime.date.today())

USUnemp = pd.DataFrame(USUnemp)
USUnemp['Date'] = USUnemp.index.values
USUnemp.insert(0, 'NumInd', range(0, 0 + len(USUnemp)))
USUnemp.set_index('NumInd', inplace=True)
USUnemp['start'] = USUnemp['Date']
USUnemp['end'] = USUnemp['Date']
USUnemp.end = USUnemp.end.shift(-1)
USUnemp.end = USUnemp['end'].fillna(datetime.date.today())

USGDP = pd.DataFrame(USGDP)
USGDP['Date'] = USGDP.index.values
USGDP.insert(0, 'NumInd', range(0, 0 + len(USGDP)))
USGDP.set_index('NumInd', inplace=True)
USGDP['start'] = USGDP['Date']
USGDP['end'] = USGDP['Date']
USGDP.end = USGDP.end.shift(-1)
USGDP.end = USGDP['end'].fillna(datetime.date.today())


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [159]:
def full_data(dataframe):
    allframes = []
    for i in dataframe.index:
        newframe = pd.DataFrame()
        newframe['Date'] = pd.date_range(dataframe.iloc[i].start, dataframe.iloc[i].end, freq = 'D')
        newframe['value'] = dataframe.iloc[i]['Value']    
        allframes.append(newframe)
    return pd.concat(allframes)


InflationCast = full_data(USInflationRate)
InflationRate = InflationCast[['Date','value']]
InflationRate['InflationRateValue'] = InflationRate['value']
InflationRate = InflationRate[['Date','InflationRateValue']]

UnempCast = full_data(USUnemp)
UnempRate = UnempCast[['Date','value']]
UnempRate['UnemploymentValue'] = UnempRate['value']
UnempRate = UnempRate[['Date','UnemploymentValue']]

GDPCast = full_data(USGDP)
GDP = GDPCast[['Date','value']]
GDP['GDP'] = GDP['value']
GDP = GDP[['Date','GDP']]


In [185]:
dataset = pd.merge(Eth, TBonds, on='Date') #Daily
dataset = pd.merge(dataset,IndexFutures, on = 'Date') # Daily
dataset = pd.merge(dataset,InflationRate, on = 'Date') # Monthly
dataset = pd.merge(dataset,UnempRate, on = 'Date') # Monthly
dataset = pd.merge(dataset,FedFundRate, on = 'Date') # Daily
dataset = pd.merge(dataset,GDP, on = 'Date') # Annually

cols = dataset.columns.tolist()
cols = cols[-1:] + cols[:-1]
dataset = dataset[cols]
dataset.GDP = dataset.GDP/10000
dataset.TBondsOpenValue = dataset.TBondsOpenValue/100
dataset.ETHOpen = dataset.ETHOpen/100
dataset.IndexFutures = dataset.IndexFutures/100
dataset.ETHHigh = dataset.ETHHigh/100
dataset.ETHLow = dataset.ETHLow/100

py.iplot([{
    'x': dataset['Date'],
    'y': dataset[col],
    'name': col
}  for col in dataset.columns], filename='dataset')

In [186]:
dataset = pd.merge(Eth, TBonds, on='Date') #Daily
dataset = pd.merge(dataset,IndexFutures, on = 'Date') # Daily
dataset = pd.merge(dataset,InflationRate, on = 'Date') # Monthly
dataset = pd.merge(dataset,UnempRate, on = 'Date') # Monthly
dataset = pd.merge(dataset,FedFundRate, on = 'Date') # Daily
dataset = pd.merge(dataset,GDP, on = 'Date') # Annually

cols = dataset.columns.tolist()
cols = cols[-1:] + cols[:-1]
dataset = dataset[cols]
dataset.GDP = dataset.GDP/1000
dataset.TBondsOpenValue = dataset.TBondsOpenValue/10
dataset.ETHOpen = dataset.ETHOpen/10
dataset.IndexFutures = dataset.IndexFutures/10
dataset.ETHHigh = dataset.ETHHigh/10
dataset.ETHLow = dataset.ETHLow/10

py.iplot([{
    'x': dataset['Date'],
    'y': dataset[col],
    'name': col
}  for col in dataset.columns], filename='dataset')

In [181]:
data = [go.Scatter(x=dataset.index, y=dataset['ETHOpen']*10)]

py.iplot(data)

In [196]:
dataset.to_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset.csv')

In [182]:
df

,close,high,low,open,time,volumefrom,volumeto,Date
0,3.00,3.00,0.6747,0.6747,1438905600,123.93,3.717900e+02,2015-08-07
1,1.20,3.00,0.1500,3.0000,1438992000,2119.43,1.438160e+03,2015-08-08
2,1.20,1.20,1.2000,1.2000,1439078400,0.00,0.000000e+00,2015-08-09
3,1.20,1.20,1.2000,1.2000,1439164800,0.00,0.000000e+00,2015-08-10
4,0.99,1.20,0.6504,1.2000,1439251200,9486.09,7.419730e+03,2015-08-11
5,1.29,1.29,0.9050,0.9900,1439337600,2140.46,2.376280e+03,2015-08-12
6,1.88,1.88,1.2600,1.2900,1439424000,3163.29,4.923220e+03,2015-08-13
7,1.79,2.10,1.7900,1.8800,1439510400,5800.73,1.107041e+04,2015-08-14
8,1.79,1.79,1.5000,1.7900,1439596800,8711.24,1.481275e+04,2015-08-15
9,1.37,1.79,0.5000,1.7900,1439683200,9714.19,1.079495e+04,2015-08-16


In [193]:
SANDP

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
Date,,,,,,,,
1982-04-21,116.35,117.60,116.05,117.45,NaN,117.45,3696.0,412.0
1982-04-22,117.00,118.40,117.00,117.90,NaN,117.90,2827.0,719.0
1982-04-23,118.40,119.75,118.25,119.65,NaN,119.65,2933.0,946.0
1982-04-26,119.30,120.60,118.65,120.55,NaN,120.55,2871.0,1084.0
1982-04-27,120.20,120.35,118.40,118.85,NaN,118.85,3854.0,1142.0
1982-04-28,118.55,119.30,117.70,118.15,NaN,118.15,3998.0,1306.0
1982-04-29,117.50,118.15,117.25,117.60,NaN,117.60,3446.0,1219.0
1982-04-30,117.55,118.40,117.40,117.50,NaN,117.50,2722.0,1295.0
1982-05-03,117.10,117.50,116.80,117.15,NaN,117.15,2695.0,1585.0


In [192]:
SANDP = quandl.get('CHRIS/CME_SP1')